# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [57]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [7]:
symbol = 'AAPL'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2239106342481,
 'week52high': 148.04,
 'week52low': 55.9,
 'week52change': 0.7217878562946246,
 'sharesOutstanding': 17462960643,
 'float': 0,
 'avg10Volume': 146757904,
 'avg30Volume': 123555532,
 'day200MovingAvg': 120.37,
 'day50MovingAvg': 132.31,
 'employees': 0,
 'ttmEPS': 3.8,
 'ttmDividendRate': 0.8388246219056904,
 'dividendYield': 0.006226077274961368,
 'nextDividendDate': '2021-01-29',
 'exDividendDate': '2020-10-27',
 'nextEarningsDate': '2021-01-24',
 'peRatio': 35.61363553803884,
 'beta': 1.1537004201368364,
 'maxChangePercent': 49.976199402024,
 'year5ChangePercent': 4.93121853245137,
 'year2ChangePercent': 2.3527462135220376,
 'year1ChangePercent': 0.7391703702036599,
 'ytdChangePercent': -0.00571302751788406,
 'month6ChangePercent': 0.25681032242202123,
 'month3ChangePercent': 0.22182184913344483,
 'month1ChangePercent': -0.005578071000050444,
 'day30ChangePercent': -0.005635347072674443,
 'day5ChangePercent': -0.0521190224003

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [8]:
data['year1ChangePercent']

0.7391703702036599

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [11]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])
#for symbol_string in symbol_strings:
    #print(symbol_string)
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [18]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['price'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A'
            ],index=my_columns),
            ignore_index=True
        )
final_dataframe   

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,122.62,0.469289,N/A
1,AAL,17.98,-0.363498,N/A
2,AAP,153.72,0.146515,N/A
3,AAPL,134.90,0.722901,N/A
4,ABBV,103.66,0.348185,N/A
...,...,...,...,...
500,YUM,106.23,-0.0223131,N/A
501,ZBH,159.73,0.0477659,N/A
502,ZBRA,398.56,0.646933,N/A
503,ZION,45.19,0.00726579,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [22]:
final_dataframe.sort_values('One-Year Price Return',ascending = False, inplace = True)
final_dataframe=final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,39.10,2.30849,N/A
1,179,FCX,26.99,1.4748,N/A
2,345,NVDA,536.68,1.25752,N/A
3,387,PYPL,244.76,1.09579,N/A
4,23,ALB,167.77,1.07762,N/A
5,24,ALGN,543.26,1.05033,N/A
6,490,WST,306.92,0.946365,N/A
7,6,ABMD,354.99,0.906926,N/A
8,388,QCOM,160.35,0.890657,N/A
9,128,DE,292.40,0.875282,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [46]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Ente the size of your portfolio:")
    
    try:
        float(portfolio_size)
    except ValueError:
        print("That is not a number! \nPlease try again")
        portfolio_size = input("Ente the size of your portfolio:")
portfolio_input()

Ente the size of your portfolio:10000000


In [47]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0,len(final_dataframe)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Price'])
final_dataframe

C:\Users\Public\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,39.10,2.30849,5115
1,179,FCX,26.99,1.4748,7410
2,345,NVDA,536.68,1.25752,372
3,387,PYPL,244.76,1.09579,817
4,23,ALB,167.77,1.07762,1192
5,24,ALGN,543.26,1.05033,368
6,490,WST,306.92,0.946365,651
7,6,ABMD,354.99,0.906926,563
8,388,QCOM,160.35,0.890657,1247
9,128,DE,292.40,0.875282,683


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [94]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score',
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            
            [
                symbol,
                data[symbol]['price'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ],index=hqm_columns),
            ignore_index = True
        )
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,122.15,N/A,0.488444,N/A,0.251842,N/A,0.186072,N/A,0.0165552,N/A,N/A
1,AAL,17.39,N/A,-0.372165,N/A,0.569454,N/A,0.532049,N/A,0.0925729,N/A,N/A
2,AAP,158.19,N/A,0.143324,N/A,-0.00362784,N/A,0.0143684,N/A,-0.0535178,N/A,N/A
3,AAPL,136.84,N/A,0.728458,N/A,0.246406,N/A,0.219947,N/A,-0.00562002,N/A,N/A
4,ABBV,104.97,N/A,0.343518,N/A,0.107789,N/A,0.226208,N/A,-0.0336898,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,106.13,N/A,-0.0223502,N/A,0.130066,N/A,0.0948823,N/A,-0.0661571,N/A,N/A
501,ZBH,160.92,N/A,0.0491978,N/A,0.149651,N/A,0.168049,N/A,-0.00286072,N/A,N/A
502,ZBRA,403.51,N/A,0.628691,N/A,0.390875,N/A,0.383245,N/A,0.00941634,N/A,N/A
503,ZION,45.52,N/A,0.00703204,N/A,0.387587,N/A,0.388003,N/A,0.0168809,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [95]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0
            
for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row,percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,122.15,N/A,0.488444,0.879208,0.251842,0.667327,0.186072,0.588119,0.0165552,0.689109,N/A
1,AAL,17.39,N/A,-0.372165,0.029703,0.569454,0.928713,0.532049,0.954455,0.0925729,0.914851,N/A
2,AAP,158.19,N/A,0.143324,0.564356,-0.00362784,0.166337,0.0143684,0.154455,-0.0535178,0.287129,N/A
3,AAPL,136.84,N/A,0.728458,0.958416,0.246406,0.651485,0.219947,0.649505,-0.00562002,0.552475,N/A
4,ABBV,104.97,N/A,0.343518,0.817822,0.107789,0.386139,0.226208,0.655446,-0.0336898,0.384158,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,106.13,N/A,-0.0223502,0.332673,0.130066,0.425743,0.0948823,0.350495,-0.0661571,0.205941,N/A
501,ZBH,160.92,N/A,0.0491978,0.433663,0.149651,0.461386,0.168049,0.562376,-0.00286072,0.570297,N/A
502,ZBRA,403.51,N/A,0.628691,0.940594,0.390875,0.821782,0.383245,0.855446,0.00941634,0.657426,N/A
503,ZION,45.52,N/A,0.00703204,0.388119,0.387587,0.817822,0.388003,0.857426,0.0168809,0.691089,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [96]:
from statistics import mean
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row,f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row,'HQM Score'] = mean(momentum_percentiles)
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,122.15,N/A,0.488444,0.879208,0.251842,0.667327,0.186072,0.588119,0.0165552,0.689109,0.705941
1,AAL,17.39,N/A,-0.372165,0.029703,0.569454,0.928713,0.532049,0.954455,0.0925729,0.914851,0.706931
2,AAP,158.19,N/A,0.143324,0.564356,-0.00362784,0.166337,0.0143684,0.154455,-0.0535178,0.287129,0.293069
3,AAPL,136.84,N/A,0.728458,0.958416,0.246406,0.651485,0.219947,0.649505,-0.00562002,0.552475,0.70297
4,ABBV,104.97,N/A,0.343518,0.817822,0.107789,0.386139,0.226208,0.655446,-0.0336898,0.384158,0.560891
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,106.13,N/A,-0.0223502,0.332673,0.130066,0.425743,0.0948823,0.350495,-0.0661571,0.205941,0.328713
501,ZBH,160.92,N/A,0.0491978,0.433663,0.149651,0.461386,0.168049,0.562376,-0.00286072,0.570297,0.506931
502,ZBRA,403.51,N/A,0.628691,0.940594,0.390875,0.821782,0.383245,0.855446,0.00941634,0.657426,0.818812
503,ZION,45.52,N/A,0.00703204,0.388119,0.387587,0.817822,0.388003,0.857426,0.0168809,0.691089,0.688614


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [97]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALB,167.410,N/A,1.09188,0.992079,0.998969,0.990099,0.781913,0.982178,0.107684,0.934653,0.974752
1,SIVB,459.570,N/A,0.827113,0.978218,0.966482,0.986139,0.516676,0.940594,0.130892,0.960396,0.966337
2,VIAC,51.970,N/A,0.490471,0.883168,0.906472,0.982178,0.723858,0.978218,0.312944,0.99604,0.959901
3,DISCA,41.383,N/A,0.432862,0.857426,0.981943,0.988119,1.04714,0.992079,0.384256,1,0.959406
4,GM,53.470,N/A,0.561479,0.910891,1.079,0.992079,0.484441,0.922772,0.222696,0.988119,0.953465
5,AMAT,102.400,N/A,0.691352,0.954455,0.535007,0.908911,0.662924,0.974257,0.124359,0.952475,0.947525
6,FCX,28.220,N/A,1.48051,0.99802,1.08448,0.994059,0.574434,0.962376,0.0353609,0.774257,0.932178
7,DISCK,34.570,N/A,0.264457,0.736634,0.853858,0.980198,0.945119,0.988119,0.351796,0.99802,0.925743
8,IVZ,21.687,N/A,0.268127,0.742574,1.10911,0.99604,0.611678,0.970297,0.186066,0.982178,0.922772
9,LLY,216.110,N/A,0.527858,0.89703,0.401443,0.829703,0.611339,0.968317,0.233761,0.990099,0.921287


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [98]:
portfolio_input()

Ente the size of your portfolio:1000000


In [99]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i,'Price'])
hqm_dataframe

C:\Users\Public\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALB,167.410,119,1.09188,0.992079,0.998969,0.990099,0.781913,0.982178,0.107684,0.934653,0.974752
1,SIVB,459.570,43,0.827113,0.978218,0.966482,0.986139,0.516676,0.940594,0.130892,0.960396,0.966337
2,VIAC,51.970,384,0.490471,0.883168,0.906472,0.982178,0.723858,0.978218,0.312944,0.99604,0.959901
3,DISCA,41.383,483,0.432862,0.857426,0.981943,0.988119,1.04714,0.992079,0.384256,1,0.959406
4,GM,53.470,374,0.561479,0.910891,1.079,0.992079,0.484441,0.922772,0.222696,0.988119,0.953465
5,AMAT,102.400,195,0.691352,0.954455,0.535007,0.908911,0.662924,0.974257,0.124359,0.952475,0.947525
6,FCX,28.220,708,1.48051,0.99802,1.08448,0.994059,0.574434,0.962376,0.0353609,0.774257,0.932178
7,DISCK,34.570,578,0.264457,0.736634,0.853858,0.980198,0.945119,0.988119,0.351796,0.99802,0.925743
8,IVZ,21.687,922,0.268127,0.742574,1.10911,0.99604,0.611678,0.970297,0.186066,0.982178,0.922772
9,LLY,216.110,92,0.527858,0.89703,0.401443,0.829703,0.611339,0.968317,0.233761,0.990099,0.921287


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [100]:
writer = pd.ExcelWriter('momentum_strategy.xlsx',engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = "Momentum Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [101]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [102]:
column_formats = {
    'A':['Ticker',string_template],
    'B':['Price',dollar_template],
    'C':['Number of Shares to Buy',integer_template],
    'D':['One-Year Price Return',percent_template],
    'E':['One-Year Return Percentile',percent_template],
    'F':['Six-Month Price Return',percent_template],
    'G':['Six-Month Return Percentile',percent_template],
    'H':['Three-Month Price Return',percent_template],
    'I':['Three-Month Return Percentile',percent_template],
    'J':['One-Month Price Return',percent_template],
    'K':['One-Month Return Percentile',percent_template],
    'L':['HQM Score',percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}',25,column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1',column_formats[column][0],column_formats[column][1])

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [103]:
writer.save()